# Chapter 3 - Exercise 2

Goal: Quadruple the size of the MNIST dataset

In [2]:
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original')
X, y = mnist['data'], mnist['target']

# Train/test split
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [3]:
from scipy.ndimage.interpolation import shift

shifted_down = shift(X_train, [1, 0])
shifted_up = shift(X_train, [-1, 0])
shifted_right = shift(X_train, [0, 1])
shifted_left = shift(X_train, [0, -1])

In [4]:
import numpy as np

augmented_data = np.concatenate([X_train, shifted_down, shifted_up, shifted_right, shifted_left])

In [5]:
y_train_augmented = np.concatenate([y_train] * 5)

In [6]:
# Shuffle the dataset
import numpy as np

shuffle_index = np.random.permutation(augmented_data.shape[0])
X_train_aug, y_train_aug = augmented_data[shuffle_index], y_train_augmented[shuffle_index]

In [7]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
%time forest_clf.fit(X_train_aug, y_train_aug)

CPU times: user 51.5 s, sys: 912 ms, total: 52.4 s
Wall time: 53.7 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [9]:
forest_clf_noaug = RandomForestClassifier(random_state=42)
%time forest_clf_noaug.fit(X_train, y_train)

CPU times: user 5.8 s, sys: 199 ms, total: 6 s
Wall time: 6.51 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [19]:
print("Random Forest no aug: ", np.mean(forest_clf_noaug.predict(X_test) == y_test))
print("Random Forest with aug: ", np.mean(forest_clf.predict(X_test) == y_test))

Random Forest no aug:  0.9455
Random Forest with aug:  0.9592


In [16]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

sgd_clf = SGDClassifier(random_state=42)
%time sgd_clf.fit(X_train, y_train)
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring='accuracy')

CPU times: user 3.8 s, sys: 221 ms, total: 4.02 s
Wall time: 4.23 s


array([ 0.86157768,  0.86489324,  0.87918188])

In [17]:
sgd_clf_aug = SGDClassifier(random_state=42)
%time sgd_clf_aug.fit(X_train_aug, y_train_aug)
cross_val_score(sgd_clf_aug, X_train_aug, y_train_aug, cv=3, scoring='accuracy')

CPU times: user 19.6 s, sys: 1.48 s, total: 21.1 s
Wall time: 22.1 s


array([ 0.83200504,  0.85133149,  0.83381335])

In [18]:
print("SGD no aug: ", np.mean(sgd_clf.predict(X_test) == y_test))
print("SGD with aug: ", np.mean(sgd_clf_aug.predict(X_test) == y_test))

SGD no aug:  0.8955
SGD with aug:  0.8799
